<a href="https://colab.research.google.com/github/gaurav-adhikari/Hand_Gesture_Detector_TF/blob/master/Hands_Gesture_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [58]:
# Imports

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [59]:
import os
for dirName, _ , signFiles in os.walk("/content/gdrive/My Drive/kaggle/SignLanguage"):
  for signFile in signFiles:
    print(os.path.join(dirName,signFile)) 

/content/gdrive/My Drive/kaggle/SignLanguage/amer_sign2.png
/content/gdrive/My Drive/kaggle/SignLanguage/amer_sign3.png
/content/gdrive/My Drive/kaggle/SignLanguage/american_sign_language.PNG
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test/sign_mnist_test.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train/sign_mnist_train.csv


In [60]:
learning_rate=0.0001

In [61]:
train=pd.read_csv("/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train/sign_mnist_train.csv")
test=pd.read_csv("/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test/sign_mnist_test.csv")

X_train=train.drop("label",axis=1)
X_test=test.drop('label',axis=1)
y_train=train['label']
y_test=test['label']

X_train = (X_train/255).values
X_test = (X_test/255).values

In [62]:
train['label'].unique()

array([ 3,  6,  2, 13, 16,  8, 22, 18, 10, 20, 17, 19, 21, 23, 24,  1, 12,
       11, 15,  4,  0,  5,  7, 14])

In [63]:
encoder=OneHotEncoder()

y_train = encoder.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_test = encoder.fit_transform(y_test.values.reshape(-1,1)).toarray()

In [64]:
((X_train.shape,y_train.shape),(X_test.shape,y_test.shape)) 

(((27455, 784), (27455, 24)), ((7172, 784), (7172, 24)))

In [65]:
def createCNNLayer(inputData,inputChannelsCount, filterCount, filterShape,poolShape, name):

  #Initialising filter
  convFilterShape=[filterShape[0],filterShape[1],inputChannelsCount,filterCount]

  weights= tf.Variable(tf.truncated_normal(convFilterShape,stddev=0.03),name=name+"_W")
  bias= tf.Variable(tf.truncated_normal([filterCount]),name=name+"_b")

  outLayer=tf.nn.conv2d(inputData,weights, [1,1,1,1],padding="SAME")

  outLayer+=bias

  outLayer = tf.nn.relu(outLayer)


  ksize = [1,poolShape[0],poolShape[1],1]

  stride=[1,2,2,1]

  outLayer=tf.nn.max_pool(outLayer,ksize,stride,padding="SAME")

  return outLayer

In [66]:
x=tf.placeholder(tf.float32,[None, 784],name="input")

X_Shaped=tf.reshape(x,[-1,28,28,1])

y=tf.placeholder(tf.float32,[None,24])

In [67]:
layer1= createCNNLayer(X_Shaped,1,32,[5,5],[2,2],"layer1")
layer2= createCNNLayer(layer1,32,64,[5,5],[2,2],"layer2")

flattenedArray=tf.reshape(layer2,[-1,7*7*64])

In [68]:
wd1 = tf.Variable(tf.truncated_normal([7*7*64,1000],stddev=0.03,name="wd1"))
bd1 = tf.Variable(tf.truncated_normal([1000],stddev=0.01,name="bd1"))
denseLayer1 = tf.nn.relu((tf.matmul(flattenedArray,wd1)+bd1))


wd2 = tf.Variable(tf.truncated_normal([1000,24],stddev=0.03,name="wd2"))
bd2 = tf.Variable(tf.truncated_normal([24],stddev=0.01,name="bd2"))
denseLayer2=tf.matmul(denseLayer1,wd2)+bd2
yhat = tf.nn.softmax(denseLayer2,name="y_")

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=denseLayer2,labels=y))

optimiser= tf.train.AdamOptimizer(learning_rate).minimize(loss)

saver=tf.train.Saver()

corr_pred= tf.equal(tf.argmax(yhat, 1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(corr_pred,tf.float32))

In [69]:
with tf.Session() as sess:

  sess.run(tf.global_variables_initializer())

  numBatches = int(len(y_train)/50)

  for epoch in range(20):
    
    cost=0
    a=0

    for i in range(numBatches):
     
      batch_x = X_train[a:a+50]
      batch_y = y_train[a:a+50]

      _ , c= sess.run([optimiser,loss],feed_dict={x:batch_x,y:batch_y})

      cost += c/numBatches
      a+=50
    

    saver.save(sess,"/content/HandModel_final.ckpt")
    
    tf.train.write_graph(sess.graph.as_graph_def(), '.', 'HandRecognitionModel.pbtxt', as_text=True)

    testDataAccuracy  = sess.run(accuracy,feed_dict={x:X_test,y:y_test})
    print("total cost at ", epoch+1," iteration ",cost," test Accuracy: ", testDataAccuracy)

  print("Training Complete!!!")

total cost at  1  iteration  3.0102530741735003  test Accuracy:  0.28778583
total cost at  2  iteration  1.5071635174620996  test Accuracy:  0.6169827
total cost at  3  iteration  0.7477047630129406  test Accuracy:  0.72685444
total cost at  4  iteration  0.43049810197075444  test Accuracy:  0.78834355
total cost at  5  iteration  0.2553459032827173  test Accuracy:  0.81929725
total cost at  6  iteration  0.14770033509082034  test Accuracy:  0.8365867
total cost at  7  iteration  0.08422515868380641  test Accuracy:  0.85373676
total cost at  8  iteration  0.049278026753748275  test Accuracy:  0.86349696
total cost at  9  iteration  0.03000241624554432  test Accuracy:  0.875488
total cost at  10  iteration  0.019464153792193434  test Accuracy:  0.88385385
total cost at  11  iteration  0.012887808773115288  test Accuracy:  0.89319575
total cost at  12  iteration  0.00841393373768357  test Accuracy:  0.9008645
total cost at  13  iteration  0.003951291330069646  test Accuracy:  0.9047685
t

In [71]:
from tensorflow.python.tools import freeze_graph

freeze_graph.freeze_graph('HandRecognitionModel.pbtxt', "", False, 
                          'HandModel_final.ckpt', "y_",
                           "save/restore_all", "save/Const:0",
                           'frozentensorflowModel.pb', True, ""  
                         )

INFO:tensorflow:Restoring parameters from HandModel_final.ckpt


INFO:tensorflow:Restoring parameters from HandModel_final.ckpt


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Converted 8 variables to const ops.


INFO:tensorflow:Converted 8 variables to const ops.


In [72]:
converter = tf.lite.TFLiteConverter.from_frozen_graph("frozentensorflowModel.pb", ["input"], ["y_"])

tflite_model = converter.convert()
open("frozentensorflowModel_final.tflite", "wb").write(tflite_model)

12854736

In [73]:
!ls

checkpoint				  HandModel_final.ckpt.index
frozentensorflowModel_final.tflite	  HandModel_final.ckpt.meta
frozentensorflowModel.pb		  HandRecognitionModel.pbtxt
gdrive					  sample_data
HandModel_final.ckpt.data-00000-of-00001
